In [1]:
%%time
#print("1")
import tensorflow as tf
from numba import cuda
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from keras.preprocessing.sequence import pad_sequences
# #print("2")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import pickle
from keras.layers import Dense, Input, Dropout
#print("3")
from keras import Sequential
#print("4")
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import keras
print(keras.__version__)
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.constraints import maxnorm
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
print("5")

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9508016033966582731
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13511798606496194581
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6439543944973650604
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15870492672
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7315827554025194689
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


Using TensorFlow backend.


2.3.1
5
CPU times: user 4.07 s, sys: 1.23 s, total: 5.3 s
Wall time: 11.9 s


/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
%%time

with open(r"../input/challengedadata/comments.txt", "rb") as f:
    clean_train_comments = pickle.load(f) 
    f.close()

with open(r"../input/challengedadata/targets.txt", "rb") as ft:
    y= pickle.load(ft) 
    ft.close()

    
y = [int(s) for s in y]


#tfidf vectorization
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

# We transform each complaint into a vector
X = tfidf.fit_transform(clean_train_comments ).toarray()
vocab_size = len(tfidf.vocabulary_) + 1
print(vocab_size)
maxlen = max([len(x) for x in X])
X_pad = pad_sequences(X, padding='post', maxlen=maxlen, dtype='float32') #https://stackoverflow.com/questions/54031161/how-can-i-get-around-keras-pad-sequences-rounding-float-values-to-zero
# end of tfidf vectorization



# standard scaler

scaler = StandardScaler(with_mean=False)
X_pad_scal = scaler.fit_transform(X_pad)
# end of scaler

120536
CPU times: user 40.9 s, sys: 5.48 s, total: 46.4 s
Wall time: 46.4 s


In [3]:
# Define the model.

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=1e-2,
        patience=2,
        verbose=1)
]

def model0(optimizer = "Adam", dropout = 0.1, init = "uniform" , dense_nparams = 256, activation = "relu",weight_constraint = 3.0):  # from https://medium.com/@am.benatmane/keras-hyperparameter-tuning-using-sklearn-pipelines-grid-search-with-cross-validation-ccfc74b0ce9f

    METRICS = [ 
          tf.keras.metrics.BinaryAccuracy(name='accuracy'),
          tf.keras.metrics.AUC(name='auc')
    ]
    
    nbr_features= 120536-1 #2500


    model = Sequential()
    model.add(Dense(dense_nparams, activation=activation, input_shape=(nbr_features,), kernel_initializer=init,  kernel_constraint=maxnorm(weight_constraint))) 
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics = METRICS) #["accuracy"] 
    return model
    

In [4]:
%%time
# model parameters

param_grid = {
"batch_size": [8,16,32,64,128],

"epochs" : [3], #[6,7,8,9,10],

"optimizer" : ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
"init" : ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
"activation" : ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'],
"dropout" : [0.1,0.2,0.3,0.4],
"dense_nparams" : [64, 128, 256, 512],
"weight_constraint" : [1, 2, 3, 4, 5]
}


# "max_features" : [2000,2500,3000,3500],
# "ngram_range" : [(1,1),(1,2),(1,3)]
# } # learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3] 
# momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9] # we don't use thes two as per docuemntation: https://keras.io/optimizers/ the default parameters of the optimizers have been set according to the respective papers, and should then not be changed.

model = KerasClassifier(build_fn=model0, verbose=10)

# Unfortunately, if i definre the model like below, i will not be able t gridsearch along keras model0's parameters, but luckily they are almost all paramreters we would like to gridsearch along, thus it will suffice to manually vary tfidf parameters each time
# estimator = Pipeline([("tfidf", TfidfVectorizer(analyzer ="word", stop_words='english')), 
#                        ('ss', StandardScaler(with_mean=False)), 
#                        ("kc", model)])

skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 1001) #5

# Model exploration

#n_jobs must be set to 1, otherwise it goses OOM
random_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, n_iter=24, scoring='roc_auc', n_jobs=1, cv=skf.split(X_pad,y), verbose=10, random_state=9, refit = "roc_auc") #24

random_search.fit(X_pad,y,  **{"callbacks": callbacks})

best_model = random_search.best_estimator_

print('\n All results:')
print(random_search.cv_results_)
print('\n Best normalized roc score for %d-fold search with %d parameter combinations:' % (5, 24))
print(random_search.best_score_ ) #*2 -1
print('\n Best hyperparameters:')
print(random_search.best_params_)
                  
# save model
joblib.dump(random_search.best_estimator_, '../working/simpleNetGrided.pkl')

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] weight_constraint=5, optimizer=Adadelta, init=zero, epochs=3, dropout=0.1, dense_nparams=128, batch_size=128, activation=relu 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/3
Epoch 2/3
Epoch 3/3
[CV]  weight_constraint=5, optimizer=Adadelta, init=zero, epochs=3, dropout=0.1, dense_nparams=128, batch_size=128, activation=relu, score=0.500, total=  11.7s
[CV] weight_constraint=5, optimizer=Adadelta, init=zero, epochs=3, dropout=0.1, dense_nparams=128, batch_size=128, activation=relu 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.7s remaining:    0.0s


Epoch 1/3
Epoch 2/3
Epoch 3/3
[CV]  weight_constraint=5, optimizer=Adadelta, init=zero, epochs=3, dropout=0.1, dense_nparams=128, batch_size=128, activation=relu, score=0.500, total=  10.6s
[CV] weight_constraint=5, optimizer=Adadelta, init=zero, epochs=3, dropout=0.1, dense_nparams=128, batch_size=128, activation=relu 


...

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[120535,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul] name: dense_227/truncated_normal/mul/

  FitFailedWarning)


[CV]  weight_constraint=2, optimizer=Adadelta, init=glorot_normal, epochs=3, dropout=0.3, dense_nparams=256, batch_size=32, activation=sigmoid, score=nan, total=  10.8s
[CV] weight_constraint=2, optimizer=Adadelta, init=glorot_normal, epochs=3, dropout=0.3, dense_nparams=256, batch_size=32, activation=sigmoid 


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
tensorflow.python.framework.errors_impl.ResourceExhaustedError: OOM when allocating tensor with shape[120535,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul] name: dense_228/truncated_normal/mul/

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 22.8min finished


[CV]  weight_constraint=2, optimizer=Adadelta, init=glorot_normal, epochs=3, dropout=0.3, dense_nparams=256, batch_size=32, activation=sigmoid, score=nan, total=  10.8s
Epoch 1/3
Epoch 2/3
Epoch 3/3

 All results:
{'mean_fit_time': array([ 9.96153245,  9.30319529,  9.3471632 ,  7.97219591, 19.37627101,
       30.20326896, 12.02630277,  8.84809918,  8.34223018,  7.84313498,
       10.32425809,  5.71627636, 10.58073983,  6.43807583,  6.79993978,
        6.93148108, 11.97241707,  7.243646  ,  6.67531109, 19.681885  ,
       11.15856142, 15.08393993, 10.93331738, 10.82110181]), 'std_fit_time': array([0.58962066, 0.14410387, 0.43046892, 0.10301881, 0.25257467,
       0.33397763, 0.07726625, 0.16586773, 0.24726137, 0.21303565,
       0.17217087, 0.25688225, 0.23713743, 0.2002824 , 0.04233426,
       0.5324426 , 0.13617156, 0.3494274 , 0.73090494, 0.21883041,
       0.1074487 , 7.40374851, 0.17118864, 0.01040821]), 'mean_score_time': array([0.62874589, 0.64007368, 0.37765193, 0.31059804, 0.45

['../working/simpleNetGrided.pkl']